In [2]:
from time import sleep
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import os
from selenium.webdriver.support.ui import WebDriverWait
import requests
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [3]:
driver = webdriver.Chrome()
# url = 'https://www.kayak.com/flights/LAX-HNL/2023-07-03/2023-07-07?sort=bestflight_a'
to_location = 'HNL'
url = 'https://www.kayak.com/flights/LAX-{to_location}/2023-07-03?sort=bestflight_a'.format(to_location = to_location)
driver.get(url)

wait = WebDriverWait(driver, 20)
for i in range(10):
    show_more_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.ULvh-button.show-more-button')))
    driver.execute_script("arguments[0].click();", show_more_button)
    try:
        wait.until(EC.staleness_of(show_more_button))
    except:
        pass

flight_rows = driver.find_elements("xpath",'//div[@class="nrc6-inner"]')
print(flight_rows)

[<selenium.webdriver.remote.webelement.WebElement (session="3d97f7223e7c389ed5aa540959ec65b0", element="70f6171d-ff37-49e4-80fe-28a702bf4ddd")>, <selenium.webdriver.remote.webelement.WebElement (session="3d97f7223e7c389ed5aa540959ec65b0", element="f69248fd-21f5-45c1-ab08-397705bf87ca")>, <selenium.webdriver.remote.webelement.WebElement (session="3d97f7223e7c389ed5aa540959ec65b0", element="8674897c-1c78-4b5b-874d-89b1cb74a961")>, <selenium.webdriver.remote.webelement.WebElement (session="3d97f7223e7c389ed5aa540959ec65b0", element="ff11fe1f-76aa-4eee-b876-1e8a2f5e7e2c")>, <selenium.webdriver.remote.webelement.WebElement (session="3d97f7223e7c389ed5aa540959ec65b0", element="a67b352a-65f4-46dd-97d9-62b0af4dc941")>, <selenium.webdriver.remote.webelement.WebElement (session="3d97f7223e7c389ed5aa540959ec65b0", element="a397894e-344d-4784-a521-8a4dbfdc9dc6")>, <selenium.webdriver.remote.webelement.WebElement (session="3d97f7223e7c389ed5aa540959ec65b0", element="9038e022-6f8b-4800-8235-31dd33fc

In [4]:
list_prices = []
list_company_names=[]
list_stops=[]
list_duration=[]

In [5]:

for WebElement in flight_rows:
    elementHTML = WebElement.get_attribute('outerHTML')
    elementSoup = BeautifulSoup(elementHTML, 'html.parser')
    # price
    temp_price = elementSoup.find("div", {"class": "nrc6-price-section"})
    price = temp_price.find("div", {"class": "f8F1-price-text"})
    list_prices.append(price.text)
    
    #company names#
    company_names = elementSoup.find("div",{"class":"c_cgF c_cgF-mod-variant-default"}).text
    list_company_names.append(company_names)
    
    # stops
    temp_stops = elementSoup.find("div", {"class": "vmXl vmXl-mod-variant-default"})
    stops = temp_stops.find("span", {"class": "JWEO-stops-text"})
    list_stops.append(stops.text)

    #durations  
    temp_durations = elementSoup.find("div", {"class": "xdW8"})
    durations = temp_durations.find("div", {"class": "vmXl vmXl-mod-variant-default"}).text
    list_duration.append(durations)

print(list_prices)
print(list_company_names)
print(list_stops)
print(list_duration)

['$326', '$319', '$400', '$400', '$400', '$400', '$450', '$490', '$409', '$556', '$471', '$481', '$481', '$481', '$471', '$680', '$471', '$362', '$481', '$481', '$481', '$555', '$555', '$485', '$481', '$485', '$411', '$481', '$485', '$471', '$471', '$471', '$515', '$481', '$539', '$455', '$539', '$515', '$481', '$539', '$539', '$481', '$539', '$539', '$539', '$539', '$555', '$539', '$395', '$411', '$539', '$539', '$548', '$555', '$395', '$539', '$485', '$515', '$481', '$485', '$481', '$485', '$411', '$426', '$498', '$539', '$435', '$548', '$481', '$485', '$471', '$539', '$486', '$426', '$555', '$469', '$539', '$438', '$539', '$411', '$481', '$459', '$343', '$486', '$539', '$481', '$411', '$426', '$545', '$450', '$539', '$395', '$539', '$485', '$459', '$343', '$395', '$539', '$411', '$545', '$537', '$395', '$397', '$400', '$688', '$411', '$400', '$561', '$498', '$616', '$395', '$545', '$600', '$481', '$688', '$555', '$564', '$689', '$459', '$375', '$438', '$568', '$481', '$343', '$545',

In [6]:
df = pd.DataFrame({
    'Price': list_prices,
    'Company Name': list_company_names,
    'Stops': list_stops,
    'Duration': list_duration
})

In [7]:
df

,Price,Company Name,Stops,Duration
0,$326,American Airlines,nonstop,5h 48m
1,$319,Alaska Airlines,nonstop,6h 10m
2,$400,United Airlines,nonstop,5h 38m
3,$400,United Airlines,nonstop,5h 30m
4,$400,United Airlines,nonstop,5h 40m
...,...,...,...,...
145,$459,Alaska Airlines,1 stop,16h 10m
146,$400,Alaska Airlines,1 stop,17h 14m
147,$720,American Airlines,2 stops,12h 00m
148,$395,United Airlines,1 stop,15h 55m


In [14]:
df = pd.read_csv('flights.csv')
print(df)
df.to_csv('flights_output.csv', index=False)

    Price       Company Name    Stops Duration
0    $326  American Airlines  nonstop   5h 48m
1    $319    Alaska Airlines  nonstop   6h 10m
2    $400    United Airlines  nonstop   5h 38m
3    $400    United Airlines  nonstop   5h 30m
4    $400    United Airlines  nonstop   5h 40m
..    ...                ...      ...      ...
145  $459    Alaska Airlines   1 stop  16h 10m
146  $400    Alaska Airlines   1 stop  17h 14m
147  $720  American Airlines  2 stops  12h 00m
148  $395    United Airlines   1 stop  15h 55m
149  $375    United Airlines   1 stop  18h 03m

[150 rows x 4 columns]
